In [6]:
import pandas as pd
import random
import copy
import os
from tqdm import tqdm

In [7]:
all_file_list = os.listdir(f".")
all_file_list = [i for i in all_file_list if i.endswith(".csv") and not i.endswith("overall.csv")]

In [8]:
all_file_list

['part_10月.csv',
 'part_5月.csv',
 'part_6月.csv',
 'part_7月.csv',
 'part_8月.csv',
 'part_9月.csv']

In [9]:
all_df = [pd.read_csv(i) for i in all_file_list]
final_df = pd.concat(all_df).reset_index(drop = True)
final_df.to_csv("first_overall.csv",index = None)
# final_df = final_df[['session_id','event_id','name']]
final_df

,user_id,stime,session_id,sequence_id,sequence_length,event_id,item_id,product_id,name,price,...,c2_id,brand_name,brand_id,item_condition_id,item_condition_name,size_name,size_id,color,shipper_id,shipper_name
0,9334,2023-10-06 03:14:08,7051a7725fb41c7f15bdb967e0bbfb3e2f035b88ff4ba3...,20231001_0,20,item_view,63317985,0_561,Genshin Impact Xiangling Harajin Mochi Kororin...,9.00,...,561.0,NaN,0,1,New,NaN,NaN,NaN,1,Buyer
1,9334,2023-10-06 03:16:05,7051a7725fb41c7f15bdb967e0bbfb3e2f035b88ff4ba3...,20231001_0,20,buy_comp,224859427,12771_463,Loungefly stitch and Angel Valentine,90.00,...,463.0,Loungefly,12771,1,New,NaN,NaN,NaN,1,Buyer
2,9334,2023-10-06 03:20:33,7051a7725fb41c7f15bdb967e0bbfb3e2f035b88ff4ba3...,20231001_0,20,item_view,58119578,12771_1152,Loungefly Disney Stitch/Angel love Cardholder,75.00,...,1152.0,Loungefly,12771,1,New,NaN,NaN,NaN,1,Buyer
3,9334,2023-10-06 03:21:02,7051a7725fb41c7f15bdb967e0bbfb3e2f035b88ff4ba3...,20231001_0,20,item_view,224859427,12771_463,Loungefly stitch and Angel Valentine,90.00,...,463.0,Loungefly,12771,1,New,NaN,NaN,NaN,1,Buyer
4,9334,2023-10-06 03:21:39,7051a7725fb41c7f15bdb967e0bbfb3e2f035b88ff4ba3...,20231001_0,20,item_view,61505053,12771_464,Loungefly Disney Lilo & stitch ducklings,130.00,...,464.0,Loungefly,12771,2,Like new,NaN,NaN,NaN,1,Buyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464977,3381363,2023-09-27 03:51:56,101673e381e1663fa80e29ea47b0befa04e3a86bf339c7...,20230901_1,17,item_view,125648658,745_2477,[OEM]I/O Shield For ROG STRIX Z390-E GAMING Mo...,7.00,...,NaN,ASUS,745,1,New,NaN,NaN,NaN,1,Buyer
464978,3381363,2023-09-27 03:52:49,101673e381e1663fa80e29ea47b0befa04e3a86bf339c7...,20230901_1,17,item_view,114669348,13899_2510,Rog strix helios,297.35,...,2510.0,MSI,13899,1,New,NaN,NaN,NaN,1,Buyer
464979,3381363,2023-09-27 03:54:44,101673e381e1663fa80e29ea47b0befa04e3a86bf339c7...,20230901_1,17,item_view,36864033,0_457,RoG Strix Go Core Gaming Headset,49.40,...,457.0,NaN,0,1,New,NaN,NaN,NaN,1,Buyer
464980,3381363,2023-09-27 03:57:04,101673e381e1663fa80e29ea47b0befa04e3a86bf339c7...,20230901_1,17,item_view,56844751,19340_968,ROG Rapture gt-AC5300 Extreme Gaming Router,255.00,...,968.0,Rapture,19340,2,Like new,NaN,NaN,NaN,1,Buyer


### preprocess with title


In [10]:
import re

final_df['size_name'] = final_df['size_name'].apply(lambda x:re.sub(r'\s*\([^)]*\)\s*', '', x)  if not pd.isna(x) else x)

print(final_df.columns)
title_lower = final_df['name'].apply(lambda x:x.lower() if not pd.isna(x) else "notnot")
cate_lower = final_df['c2_name'].apply(lambda x:x.lower() if not pd.isna(x) else "notnot")
brand_lower = final_df['brand_name'].apply(lambda x:x.lower() if not pd.isna(x) else "notnot")
size_lower = final_df['size_name'].apply(lambda x:x.lower() if not pd.isna(x) else "notnot")
color_lower = final_df['color'].apply(lambda x:x.lower() if not pd.isna(x) else "notnot")
cate_not_in = [(i not in j) for i,j in zip(cate_lower,title_lower)]
brand_not_in = [(i not in j) for i,j in zip(brand_lower,title_lower)]
size_not_in = [(i not in j) for i,j in zip(size_lower,title_lower)]
color_not_in = [(i not in j) for i,j in zip(color_lower,title_lower)]

title = final_df['name'].tolist()
cate = final_df['c2_name'].tolist()
brand = final_df['brand_name'].tolist()
size = final_df['size_name'].tolist()
color = final_df['color'].tolist()
row_idx = 0
renew_title_list = []
for i,j,k,w,q in zip(title,cate,brand,size,color):
    i = i.split(" ")
    range_len = len(i)
    if cate_not_in[row_idx] == True and cate_lower[row_idx] != "notnot":
        j_idx = random.randint(0, range_len)
        i.insert(j_idx, j)
    if brand_not_in[row_idx] == True and brand_lower[row_idx] != "notnot":
        k_idx = random.randint(0, range_len)
        i.insert(k_idx, k)
    if size_not_in[row_idx] == True and size_lower[row_idx] != "notnot":
        w_idx = random.randint(0, range_len)
        i.insert(w_idx, w)
    if color_not_in[row_idx] == True and color_lower[row_idx] != "notnot":
        q_idx = random.randint(0, range_len)
        i.insert(q_idx, q)
    renew_title_list.append(" ".join(i))
    row_idx += 1

final_df['name'] = renew_title_list

Index(['user_id', 'stime', 'session_id', 'sequence_id', 'sequence_length',
       'event_id', 'item_id', 'product_id', 'name', 'price', 'c0_name',
       'c0_id', 'c1_name', 'c1_id', 'c2_name', 'c2_id', 'brand_name',
       'brand_id', 'item_condition_id', 'item_condition_name', 'size_name',
       'size_id', 'color', 'shipper_id', 'shipper_name'],
      dtype='object')


### Sample

In [11]:
def generate_number():
    return random.randint(3, 10)
#     if random.random() < 1/3:
#         return random.randint(3, 6)
#     else:
#         return random.randint(7, 10)
final_df_copy = copy.deepcopy(final_df).reset_index(drop = True)
final_df_copy['stime'] = pd.to_datetime(final_df_copy['stime'])
final_df_copy = final_df_copy.sort_values(['session_id','stime']).reset_index(drop = True)
all_sess_order = final_df_copy.drop_duplicates("session_id")['session_id']
all_label_idx_list = []
sess_times_dict = final_df_copy.groupby("session_id").size().to_dict()
for single_sess in tqdm(all_sess_order):
    cur_sess_len = sess_times_dict[single_sess]
    rand_num = int( (generate_number() / 10) * (cur_sess_len - 1) )
    cur_label_idx_list = ["no" for i in range(cur_sess_len)]
    cur_label_idx_list[rand_num] = "label"
#     flag = 0
    for i in range(0, rand_num):
        if random.random() < 3/4: #  or flag == 1
            cur_label_idx_list[i] = "sample" 
#             flag = 0
        else:
            cur_label_idx_list[i] = "query" 
#             flag = 1

    all_label_idx_list += cur_label_idx_list
final_df_copy['is_label'] = all_label_idx_list
final_df_copy = final_df_copy[final_df_copy['is_label'] != "no"]
print(f"截断之前：{len(final_df_copy)}")
final_df_limit = final_df_copy.groupby('session_id').tail(12).reset_index(drop = True)
print(f"截断之后：{len(final_df_limit)}")
final_df_limit

# cut short sequence
tmp = (final_df_limit.groupby("session_id").size() >= 3).reset_index(drop = False)
tmp.columns = ['session_id','no_filter']
tmp = pd.merge(final_df_limit[['session_id']], tmp, how = "left", on = "session_id")
final_df_limit = final_df_limit[tmp['no_filter'] == True].reset_index(drop = True)
final_df_limit

100%|████████████████████████████████████████████████████████████████████████| 30512/30512 [00:00<00:00, 240245.93it/s]


截断之前：302287
截断之后：218915


,user_id,stime,session_id,sequence_id,sequence_length,event_id,item_id,product_id,name,price,...,brand_name,brand_id,item_condition_id,item_condition_name,size_name,size_id,color,shipper_id,shipper_name,is_label
0,2854198,2023-09-06 17:33:27,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,20230901_1,22,item_view,181856567,22105_3029,Satin shawl collar SHEIN Pajama sets allover p...,8.00,...,SHEIN,22105,2,Like new,S,5.0,NaN,1,Buyer,query
1,2854198,2023-09-06 17:34:59,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,20230901_1,22,item_view,232226429,4815_159,Girls 0-24 mos Girls 9-12 Months Carter's dress,4.75,...,Carter's,4815,1,New,9-12 Months,157.0,NaN,1,Buyer,query
2,2854198,2023-09-06 17:35:26,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,20230901_1,22,item_view,204577100,0_3029,Lonxu Pink With Black Trim Silk Satin Pajama P...,7.00,...,NaN,0,1,New,S,5.0,NaN,1,Buyer,sample
3,2854198,2023-09-06 17:35:39,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,20230901_1,22,item_add_to_cart_tap,204577100,0_3029,Lonxu Pink With Black Trim Pajama sets Silk Sa...,7.00,...,NaN,0,1,New,S,5.0,NaN,1,Buyer,sample
4,2854198,2023-09-06 17:36:00,000a35550721dae62ddbd1a52bf28631a30162f3a1f005...,20230901_1,22,buy_comp,204577100,0_3029,Pajama sets Lonxu Pink With Black Trim Silk Sa...,7.00,...,NaN,0,1,New,S,5.0,NaN,1,Buyer,sample
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215098,1752656,2023-07-29 01:22:40,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,20230701_4,22,item_view,60111473,4089_171,Girls 2T-5T 5t Custom Variety Pack Girls Bundle,33.25,...,Custom Variety Pack,4089,2,Like new,5T,164.0,NaN,1,Buyer,sample
215099,1752656,2023-07-29 01:24:42,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,20230701_4,22,item_view,223708013,4089_170,Girls XS 4/5 T Custom Variety Pack long Girls ...,7.00,...,Custom Variety Pack,4089,1,New,XS,170.0,NaN,1,Buyer,sample
215100,1752656,2023-07-29 01:25:28,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,20230701_4,22,item_add_to_cart_tap,223708013,4089_170,XS Custom Variety Pack Girls 4/5 T long sleeve...,7.00,...,Custom Variety Pack,4089,1,New,XS,170.0,NaN,1,Buyer,sample
215101,1752656,2023-07-29 01:25:55,fffc7a2257c114a4fbad5e1a4f5ec7ed1d840286b1d9ce...,20230701_4,22,item_view,85942818,0_3028,Nightgowns & sleep shirts One Size Sleep shirt,5.00,...,NaN,0,2,Like new,One Size,50.0,NaN,1,Buyer,sample


In [12]:
# check
final_df_limit[final_df_limit['is_label'] == 'label']['session_id'].value_counts()

0787865c2fa660d23b4cc58a733abbfd71e7bbcf7bff2dcc20796ce46d8cd7dd    1
75b7a34c4283fe7ae4f799d4aa39dabb199114e963cf218986e65b1cc015567b    1
b6321da8729e733ab17d47a542b7cc8cc03cf780698eb163ed3b13e54c2224a0    1
f002fd494757fdbd37381de6272a82eaf404574bf5a98ea714b2555cc74671eb    1
c8b47390d834ba41d51817fd644e798bb72a768b0492725308494702bb0f3a61    1
                                                                   ..
6a4fc7492c0def8a3448b77a3e8349c8e8a5ef298e6c720b492cfb933842dbf6    1
ec1b0c158167e808a73ce442f5723b8059322051e44411b83ac43be4b9a1c99e    1
d8f137e74387a8ad977c1b716b5af19a1c4daa2c206f5dd80ea302d92c1bf523    1
34687c169fc0a146d3f81e3db9e0c12a221e083151b00bc1291d32e2d421023e    1
0e0776e54e385901e0dc2b5693dd30ed468e2e76f3c7d3274667f90d39296348    1
Name: session_id, Length: 28569, dtype: int64

In [13]:
final_df_limit.reset_index(drop = True).reset_index(drop = False)[['index','session_id','event_id','name','is_label']].to_csv("next_title_gen数据生成/行为序列数据.csv",index = None)